In [ ]:
import os
import pandas as pd

In [ ]:
resnet_df = pd.read_pickle('resnet50_0_preds.csv')
efficientnet_df = pd.read_pickle('efficientnetb4_0_preds.csv')

In [ ]:
def download(images, target_path):
    resnet_server = 'ubuntu@ec2-3-250-23-160.eu-west-1.compute.amazonaws.com'
    efficientnet_server = 'ubuntu@ec2-54-195-189-117.eu-west-1.compute.amazonaws.com'
    masks_path = '/home/ubuntu/store/masks/masks_resized/'
    resnet_gradcam_path = '/home/ubuntu/store/resnet-final-size/visualisation/gradcam/resnet50_0/'
    efficientnet_gradcam_path = '/home/ubuntu/store/efficientnet-final-size/visualisation/gradcam/efficientnetb4_0/'
    
    for image in images:
        image = image.replace(' ', '\ ')
        # Get base image
        print(f"scp \"{resnet_server}:/home/ubuntu/store/DermX-test-set/test/{image}\" {target_path}")
        print(os.system(f"scp \"{resnet_server}:/home/ubuntu/store/DermX-test-set/test/{image}\" {target_path}"))
        
        # Get ResNet GradCAM for the GT class
        gradcam_filename = image.replace('.jpeg', '').split('/')
        gradcam_filename = gradcam_filename[1] + '_' + gradcam_filename[0] + '.png'
        print(f"scp \"{resnet_server}:{resnet_gradcam_path}{gradcam_filename}\" {target_path}")
        print(os.system(f"scp \"{resnet_server}:{resnet_gradcam_path}{gradcam_filename}\" {target_path}resnet/"))
        
        # Get EfficientNet GradCAM for the GT class
        print(f"scp \"{efficientnet_server}:{efficientnet_gradcam_path}{gradcam_filename}\" {target_path}")
        print(os.system(f"scp \"{efficientnet_server}:{efficientnet_gradcam_path}{gradcam_filename}\" {target_path}efficientnet/"))
        
        # Get derm masks
        print(f"scp \"{resnet_server}:{masks_path}{image.split('/')[1].replace('.jpeg', '')}*\" {target_path}")
        print(os.system(f"scp \"{resnet_server}:{masks_path}{image.split('/')[1].replace('.jpeg', '')}*\" {target_path}"))

In [ ]:
full_df = resnet_df.merge(efficientnet_df, left_on='filenames', right_on='filenames', suffixes=('_resnet', '_efficientnet'))
full_df

In [ ]:
both_correct = full_df[(full_df['actual_resnet'] == full_df['pred_resnet']) & (full_df['actual_resnet'] == full_df['pred_efficientnet'])]
both_correct

In [ ]:
both_correct.groupby('actual_resnet').first()

,pred_resnet,filenames,actual_efficientnet,pred_efficientnet
actual_resnet,,,,
0,0,Acne/032879HB.jpeg,0,0
1,1,Actinic keratosis/005103HB.jpeg,1,1
2,2,Psoriasis/043446HB.jpeg,2,2
3,3,Seborrheic dermatitis/042355HB.jpeg,3,3
5,5,Vitiligo/10489-Vitiligo.jpeg,5,5


In [ ]:
download(both_correct.groupby('actual_resnet').first()['filenames'].values, '/mnt/c/Users/raluca/Downloads/appendix_pictures/both/')

In [ ]:
resnet_correct = full_df[(full_df['actual_resnet'] == full_df['pred_resnet']) & (full_df['actual_resnet'] != full_df['pred_efficientnet'])]
resnet_correct

In [ ]:
resnet_correct.groupby('actual_resnet').first()

,pred_resnet,filenames,actual_efficientnet,pred_efficientnet
actual_resnet,,,,
0,0,Acne/033370HB.jpeg,0,1
1,1,Actinic keratosis/016730HB.jpeg,1,2
2,2,Psoriasis/facial-psoriasis12--WatermarkedWyJXY...,2,3
3,3,Seborrheic dermatitis/042932HB.jpeg,3,2
5,5,Vitiligo/041357HB.jpeg,5,2


In [ ]:
download(resnet_correct.groupby('actual_resnet').first()['filenames'].values, '/mnt/c/Users/raluca/Downloads/appendix_pictures/resnet/')

In [ ]:
efficientnet_correct = full_df[(full_df['actual_resnet'] != full_df['pred_resnet']) & (full_df['actual_resnet'] == full_df['pred_efficientnet'])]
efficientnet_correct

In [ ]:
efficientnet_correct.groupby('actual_resnet').first()

,pred_resnet,filenames,actual_efficientnet,pred_efficientnet
actual_resnet,,,,
0,3,Acne/032439HB.jpeg,0,0
1,3,Actinic keratosis/005102HB.jpeg,1,1
2,3,Psoriasis/044567HB.jpeg,2,2
3,2,Seborrheic dermatitis/043023HB.jpeg,3,3
4,2,Viral warts/10427-DSC03079.jpeg,4,4


In [ ]:
download(efficientnet_correct.groupby('actual_resnet').first()['filenames'].values, '/mnt/c/Users/raluca/Downloads/appendix_pictures/efficientnet/')

In [ ]:
none_correct = full_df[(full_df['actual_resnet'] != full_df['pred_resnet']) & (full_df['actual_resnet'] != full_df['pred_efficientnet'])]
none_correct

,actual_resnet,pred_resnet,filenames,actual_efficientnet,pred_efficientnet
3,0,5,Acne/032720HB.jpeg,0,2
10,0,1,Acne/033353HB.jpeg,0,2
15,0,2,Acne/033751HB.jpeg,0,2
19,0,2,Acne/033877HB.jpeg,0,2
20,0,2,Acne/033899HB.jpeg,0,2
...,...,...,...,...,...
561,5,4,Vitiligo/vitiligo-66--WatermarkedWyJXYXRlcm1hc...,5,4
562,5,0,Vitiligo/vitiligo1--WatermarkedWyJXYXRlcm1hcmt...,5,2
563,5,4,Vitiligo/vitiligo2--WatermarkedWyJXYXRlcm1hcmt...,5,4
564,5,2,Vitiligo/vitiligo3--WatermarkedWyJXYXRlcm1hcmt...,5,2


In [ ]:
none_correct.groupby('actual_resnet').first()

,pred_resnet,filenames,actual_efficientnet,pred_efficientnet
actual_resnet,,,,
0,5,Acne/032720HB.jpeg,0,2
1,3,Actinic keratosis/000498HB.jpeg,1,2
2,3,Psoriasis/10146-Psoriasis-202.jpeg,2,0
3,1,Seborrheic dermatitis/042964HB.jpeg,3,2
4,5,Viral warts/041854VB.jpeg,4,3
5,1,Vitiligo/041355HB.jpeg,5,2


In [ ]:
download(none_correct.groupby('actual_resnet').first()['filenames'].values, '/mnt/c/Users/raluca/Downloads/appendix_pictures/none/')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm 
from PIL import Image
import numpy as np
from pathlib import Path

In [ ]:
def derm_masks_overlay(image_path, masks_paths, alpha=0.4):
    image = Image.open(image_path).convert("RGBA")
    cm = plt.get_cmap('jet')
    union_mask = np.zeros((300, 400))
    for mask_path in masks_paths:
        mask = Image.open(mask_path).resize((400, 300))
        union_mask = union_mask + mask
    union_mask = union_mask * (1 / (union_mask.max()))
    union_mask = cm(union_mask)
    union_mask = Image.fromarray(np.uint8(union_mask * 255)).convert('RGBA')
    union_mask.putalpha(int(alpha * 255))
    # Superimpose the heatmap on original image
    image.paste(union_mask, (0, 0), union_mask)
    return image

In [ ]:
import glob

In [ ]:
images_path = Path('/mnt/c/Users/raluca/Downloads/appendix_pictures/both/')

images = both_correct.groupby('actual_resnet').first()['filenames'].values
fig, axs = plt.subplots(len(images), 4, figsize=(15, 15))
fig.tight_layout()

for idx, image in enumerate(images):
    image_name = image.split('/')[1].replace('.jpeg', '')
    print(image_name)
    masks_paths = glob.glob(str(images_path) + "/" + image_name + "_*")
    efficientnet_gradcam_path = glob.glob(str(images_path / 'efficientnet') + "/" + image_name + "_*")[0]
    resnet_gradcam_path = glob.glob(str(images_path / 'resnet') + "/" + image_name + "_*")[0]
    derm_mask = derm_masks_overlay(images_path / (image_name + ".jpeg"), masks_paths)
    
    axs[idx, 0].imshow(Image.open(images_path / (image_name + '.jpeg')))
    axs[idx, 1].imshow(derm_mask)
    axs[idx, 2].imshow(Image.open(efficientnet_gradcam_path))
    axs[idx, 3].imshow(Image.open(resnet_gradcam_path))


for ax in axs.flat:
    ax.set(xticks=[], yticks=[])
    
xs = ['Image', 'Dermatologist map', 'EfficientNet GradCAM', 'ResNet GradCAM']
for idx, ax in enumerate(axs[0, :]):
    ax.set(xlabel=xs[idx])
    ax.xaxis.set_label_position('top') 
    
ys = ['Acne', 'Actinic keratosis', 'Psoriasis', 'Seborrheic dermatitis', 'Vitiligo']
for idx, ax in enumerate(axs[:, 0]):
    ax.set(ylabel=ys[idx])

fig.savefig('both.png')

In [ ]:
images_path = Path('/mnt/c/Users/raluca/Downloads/appendix_pictures/none/')

images = none_correct.groupby('actual_resnet').first()['filenames'].values
fig, axs = plt.subplots(len(images), 4, figsize=(15, 15))
fig.tight_layout()

for idx, image in enumerate(images):
    image_name = image.split('/')[1].replace('.jpeg', '')
    print(image_name)
    masks_paths = glob.glob(str(images_path) + "/" + image_name + "_*")
    efficientnet_gradcam_path = glob.glob(str(images_path / 'efficientnet') + "/" + image_name + "_*")[0]
    resnet_gradcam_path = glob.glob(str(images_path / 'resnet') + "/" + image_name + "_*")[0]
    derm_mask = derm_masks_overlay(images_path / (image_name + ".jpeg"), masks_paths)
    
    axs[idx, 0].imshow(Image.open(images_path / (image_name + '.jpeg')))
    axs[idx, 1].imshow(derm_mask)
    axs[idx, 2].imshow(Image.open(efficientnet_gradcam_path))
    axs[idx, 3].imshow(Image.open(resnet_gradcam_path))


for ax in axs.flat:
    ax.set(xticks=[], yticks=[])
    
xs = ['Image', 'Dermatologist map', 'EfficientNet GradCAM', 'ResNet GradCAM']
for idx, ax in enumerate(axs[0, :]):
    ax.set(xlabel=xs[idx])
    ax.xaxis.set_label_position('top') 
    
ys = ['Acne', 'Actinic keratosis', 'Psoriasis', 'Seborrheic dermatitis', 'Viral warts', 'Vitiligo']
for idx, ax in enumerate(axs[:, 0]):
    ax.set(ylabel=ys[idx])

fig.savefig('none.png')

In [ ]:
images_path = Path('/mnt/c/Users/raluca/Downloads/appendix_pictures/efficientnet/')

images = efficientnet_correct.groupby('actual_resnet').first()['filenames'].values
fig, axs = plt.subplots(len(images), 4, figsize=(15, 15))
fig.tight_layout()

for idx, image in enumerate(images):
    image_name = image.split('/')[1].replace('.jpeg', '')
    print(image_name)
    masks_paths = glob.glob(str(images_path) + "/" + image_name + "_*")
    efficientnet_gradcam_path = glob.glob(str(images_path / 'efficientnet') + "/" + image_name + "_*")[0]
    resnet_gradcam_path = glob.glob(str(images_path / 'resnet') + "/" + image_name + "_*")[0]
    derm_mask = derm_masks_overlay(images_path / (image_name + ".jpeg"), masks_paths)
    
    axs[idx, 0].imshow(Image.open(images_path / (image_name + '.jpeg')))
    axs[idx, 1].imshow(derm_mask)
    axs[idx, 2].imshow(Image.open(efficientnet_gradcam_path))
    axs[idx, 3].imshow(Image.open(resnet_gradcam_path))


for ax in axs.flat:
    ax.set(xticks=[], yticks=[])
    
xs = ['Image', 'Dermatologist map', 'EfficientNet GradCAM', 'ResNet GradCAM']
for idx, ax in enumerate(axs[0, :]):
    ax.set(xlabel=xs[idx])
    ax.xaxis.set_label_position('top') 
    
ys = ['Acne', 'Actinic keratosis', 'Psoriasis', 'Seborrheic dermatitis', 'Viral warts']
for idx, ax in enumerate(axs[:, 0]):
    ax.set(ylabel=ys[idx])

fig.savefig('efficientnet.png')

In [ ]:
images_path = Path('/mnt/c/Users/raluca/Downloads/appendix_pictures/resnet/')

images = resnet_correct.groupby('actual_resnet').first()['filenames'].values
fig, axs = plt.subplots(len(images), 4, figsize=(15, 15))
fig.tight_layout()

for idx, image in enumerate(images):
    image_name = image.split('/')[1].replace('.jpeg', '')
    print(image_name)
    masks_paths = glob.glob(str(images_path) + "/" + image_name + "_*")
    efficientnet_gradcam_path = glob.glob(str(images_path / 'efficientnet') + "/" + image_name + "_*")[0]
    resnet_gradcam_path = glob.glob(str(images_path / 'resnet') + "/" + image_name + "_*")[0]
    derm_mask = derm_masks_overlay(images_path / (image_name + ".jpeg"), masks_paths)
    
    axs[idx, 0].imshow(Image.open(images_path / (image_name + '.jpeg')))
    axs[idx, 1].imshow(derm_mask)
    axs[idx, 2].imshow(Image.open(efficientnet_gradcam_path))
    axs[idx, 3].imshow(Image.open(resnet_gradcam_path))


for ax in axs.flat:
    ax.set(xticks=[], yticks=[])
    
xs = ['Image', 'Dermatologist map', 'EfficientNet GradCAM', 'ResNet GradCAM']
for idx, ax in enumerate(axs[0, :]):
    ax.set(xlabel=xs[idx])
    ax.xaxis.set_label_position('top') 
    
ys = ['Acne', 'Actinic keratosis', 'Psoriasis', 'Seborrheic dermatitis', 'Vitiligo']
for idx, ax in enumerate(axs[:, 0]):
    ax.set(ylabel=ys[idx])

fig.savefig('resnet.png')